In [2]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback

In [14]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


def add_column(tname):
    c = """
    ALTER TABLE wdpa_%s_intersections 
    ADD COLUMN species_area double precision
    ;
    """
    c = c % (tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def update_table(tname):
    c = """
    UPDATE wdpa_%s_intersections
    SET species_area = m.sum
    FROM %s_areas as m
    where 
    wdpa_%s_intersections.%s_gid = m.%s_gid
    ;
    """
    c = c % (tname, tname, tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def rename_column(tname):
    c = """
    ALTER TABLE wdpa_%s_intersections 
    RENAME COLUMN species_area TO %s_gid_area;    
    """
    c = c % (tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            
    

In [9]:
tnames = []
for f in glob.glob("../../data/redlist/*/*.shp"):
    name = os.path.basename(f).split(".")[0]
    tnames.append(name.lower())
tnames.sort()

In [ ]:
for tname in tnames:
    add_column(tname)

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=16)
for tname in tnames:
    exe.submit(update_table, tname)
exe.shutdown()
None

In [15]:
exe = SimpleProcessPoolExecutor(max_workers=16)
for tname in tnames:
    exe.submit(rename_column, tname)
exe.shutdown()
None

ERROR:  column "species_area" does not exist
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
SimpleProcessPoolExecutor succeeded: all 32 jobs completed
